# Inicializace

In [27]:
from collections import deque               # Ukladani stavu
from __future__ import division             # Deleni realnych cisel (kvuli verzi Pythonu 2.6)
import sys                                  # Pro navazani knihovny Open AI Gym
import numpy as np                          # Knihovna pro operace s maticemi
sys.path.append('/home/xbucha02/libraries') # Adresa knihovny Open AI Gym
import gym                                  # Knihovna Open AI Gym
#from gym import wrappers                    # Pomucka na nahravani
env = gym.make('CartPole-v0')            # Konkretni hra z Open AI Gym
#env = wrappers.Monitor(env, '/home/lachubcz/tmp/cartpole-experiment-1', force=True) #Nahravani zaznamu
#gpuMemoryUsage=1                            # Vyuziti pameti GPU
#import tensorflow as tf                     # Knihovna TensorFlow pro sdileni GPU
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = gpuMemoryUsage
#set_session(tf.Session(config=config))
import random

# Parametry
observetime = 500                          # Pocet framu kazde hry
startEpsilon = 1.0                         # Pravdepodobnost konani nahodneho tahu na zacatku
endEpsilon = 0.1                           # Pravdepodobnost konani nahodneho tahu na konci
currentEpsilon = startEpsilon              # Soucasna pravdepodobnost konani nahodneho tahu
epsilonDiminution = 0.995                  # Hodnota snizovani epsilonu
gamma = 0.95                               # Discount faktor
minibatchSize = 32                         # Velikost minibatche
episodes = 5000                            # Pocet epizod
actionCount = env.action_space.n           # Pocet vstupu do prostredi
stateSize = env.observation_space.shape[0] # Pocet vystupu z prostredi
learningRate = 0.001                       # Learning rate
memorySize = 10000                         # Velikost Replay memory

# Model NN

In [28]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics
from keras import backend as K

def _huber_loss(target, prediction):
    # sqrt(1+error^2)-1
    error = prediction - target
    return K.mean(K.sqrt(1+K.square(error))-1, axis=-1)

def getDQN(actionCount,stateSize):
    net = Sequential()
    net.add(Dense(24, activation="relu", input_dim=stateSize))
    net.add(Dense(24, activation="relu"))
    net.add(Dense(actionCount, activation="linear"))
    
    net.summary()
    
    net.compile(loss=_huber_loss, optimizer=optimizers.Adam(lr=learningRate), metrics=['accuracy'])
    
    return net

def loadDQN(net, name):
    net.load_weights(name)

def saveDQN(net, name):
    net.save_weights(name)

# Algoritmus DQN

In [29]:
net = getDQN(actionCount, stateSize)
netTarget = getDQN(actionCount, stateSize)
observation = env.reset()                    # Reset prostredi
memory = deque(maxlen=memorySize)

global scores
scores = []
global episodes
episodesList = []

for eps in range (episodes):
    
    #D = deque() #vyprazdneni D
    state = env.reset()
    state = np.reshape(state, [1, stateSize])
    #env.render()

    #q = 0
    for t in range(observetime):
        #if q == 0:
            
        #else:
        #    if q == 4:
        #        q = 0
        #    else:
        #        q = q + 1
        if np.random.rand() <= currentEpsilon:
            action = np.random.randint(0, actionCount, size=1)[0]
        else:
            Q = net.predict(state)

            action = np.argmax(Q)

        observation_new, reward, done, info = env.step(int(action))
        
        reward = reward if not done else -10
        
        next_state = np.reshape(observation_new, [1, stateSize])

        memory.append((state, action, reward, next_state, done))

        state = next_state         # Update state

        if done:
            print("Episode: {}/{}, epsilon: {:.2}, score: {}".format(eps, episodes, currentEpsilon, t))
            scores.append(t)
            episodesList.append(eps)
            netTarget.set_weights(net.get_weights())
            break
            
    if eps > 0:
        if len(memory) < minibatchSize:
            continue
        minibatch = random.sample(memory, minibatchSize) #z D vybere pocet mb_size samplu

        for i in range(0, minibatchSize):

            stateMb = minibatch[i][0]
            actionMb = minibatch[i][1]
            rewardMb = minibatch[i][2]
            state_newMb = minibatch[i][3]
            doneMb = minibatch[i][4]
            
            target_f = net.predict(stateMb)
            
            if doneMb:
                target_f[0][actionMb] = targetMb
            else:
                a = net.predict(state_newMb)[0]
                t = netTarget.predict(state_newMb)[0]
                target_f[0][actionMb] = rewardMb + gamma * t[np.argmax(a)]

            net.fit(stateMb, target_f, epochs=1, verbose=0)

        if currentEpsilon > endEpsilon:
            currentEpsilon = currentEpsilon * epsilonDiminution

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 24)                120       
_________________________________________________________________
dense_38 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_39 (Dense)             (None, 2)                 50        
Total params: 770.0
Trainable params: 770
Non-trainable params: 0.0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 24)                120       
_________________________________________________________________
dense_41 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_42

Episode: 157/5000, epsilon: 0.46, score: 8
Episode: 158/5000, epsilon: 0.46, score: 11
Episode: 159/5000, epsilon: 0.45, score: 12
Episode: 160/5000, epsilon: 0.45, score: 16
Episode: 161/5000, epsilon: 0.45, score: 14
Episode: 162/5000, epsilon: 0.45, score: 36
Episode: 163/5000, epsilon: 0.44, score: 17
Episode: 164/5000, epsilon: 0.44, score: 19
Episode: 165/5000, epsilon: 0.44, score: 9
Episode: 166/5000, epsilon: 0.44, score: 19
Episode: 167/5000, epsilon: 0.44, score: 9
Episode: 168/5000, epsilon: 0.43, score: 8
Episode: 169/5000, epsilon: 0.43, score: 10
Episode: 170/5000, epsilon: 0.43, score: 11
Episode: 171/5000, epsilon: 0.43, score: 11
Episode: 172/5000, epsilon: 0.42, score: 19
Episode: 173/5000, epsilon: 0.42, score: 11
Episode: 174/5000, epsilon: 0.42, score: 10
Episode: 175/5000, epsilon: 0.42, score: 13
Episode: 176/5000, epsilon: 0.42, score: 10
Episode: 177/5000, epsilon: 0.41, score: 8
Episode: 178/5000, epsilon: 0.41, score: 15
Episode: 179/5000, epsilon: 0.41, sco

Episode: 346/5000, epsilon: 0.18, score: 12
Episode: 347/5000, epsilon: 0.18, score: 7
Episode: 348/5000, epsilon: 0.18, score: 12
Episode: 349/5000, epsilon: 0.17, score: 11
Episode: 350/5000, epsilon: 0.17, score: 12
Episode: 351/5000, epsilon: 0.17, score: 9
Episode: 352/5000, epsilon: 0.17, score: 8
Episode: 353/5000, epsilon: 0.17, score: 8
Episode: 354/5000, epsilon: 0.17, score: 9
Episode: 355/5000, epsilon: 0.17, score: 8
Episode: 356/5000, epsilon: 0.17, score: 13
Episode: 357/5000, epsilon: 0.17, score: 9
Episode: 358/5000, epsilon: 0.17, score: 8
Episode: 359/5000, epsilon: 0.17, score: 9
Episode: 360/5000, epsilon: 0.17, score: 9
Episode: 361/5000, epsilon: 0.16, score: 8
Episode: 362/5000, epsilon: 0.16, score: 8
Episode: 363/5000, epsilon: 0.16, score: 9
Episode: 364/5000, epsilon: 0.16, score: 9
Episode: 365/5000, epsilon: 0.16, score: 9
Episode: 366/5000, epsilon: 0.16, score: 10
Episode: 367/5000, epsilon: 0.16, score: 9
Episode: 368/5000, epsilon: 0.16, score: 9
Episo

Episode: 538/5000, epsilon: 0.1, score: 9
Episode: 539/5000, epsilon: 0.1, score: 9
Episode: 540/5000, epsilon: 0.1, score: 9
Episode: 541/5000, epsilon: 0.1, score: 8
Episode: 542/5000, epsilon: 0.1, score: 10
Episode: 543/5000, epsilon: 0.1, score: 8
Episode: 544/5000, epsilon: 0.1, score: 9
Episode: 545/5000, epsilon: 0.1, score: 7
Episode: 546/5000, epsilon: 0.1, score: 8
Episode: 547/5000, epsilon: 0.1, score: 9
Episode: 548/5000, epsilon: 0.1, score: 9
Episode: 549/5000, epsilon: 0.1, score: 7
Episode: 550/5000, epsilon: 0.1, score: 10
Episode: 551/5000, epsilon: 0.1, score: 10
Episode: 552/5000, epsilon: 0.1, score: 8
Episode: 553/5000, epsilon: 0.1, score: 7
Episode: 554/5000, epsilon: 0.1, score: 9
Episode: 555/5000, epsilon: 0.1, score: 9
Episode: 556/5000, epsilon: 0.1, score: 8
Episode: 557/5000, epsilon: 0.1, score: 9
Episode: 558/5000, epsilon: 0.1, score: 7
Episode: 559/5000, epsilon: 0.1, score: 8
Episode: 560/5000, epsilon: 0.1, score: 10
Episode: 561/5000, epsilon: 0.

Episode: 732/5000, epsilon: 0.1, score: 9
Episode: 733/5000, epsilon: 0.1, score: 11
Episode: 734/5000, epsilon: 0.1, score: 9
Episode: 735/5000, epsilon: 0.1, score: 8
Episode: 736/5000, epsilon: 0.1, score: 9
Episode: 737/5000, epsilon: 0.1, score: 9
Episode: 738/5000, epsilon: 0.1, score: 9
Episode: 739/5000, epsilon: 0.1, score: 7
Episode: 740/5000, epsilon: 0.1, score: 7
Episode: 741/5000, epsilon: 0.1, score: 8
Episode: 742/5000, epsilon: 0.1, score: 11
Episode: 743/5000, epsilon: 0.1, score: 9
Episode: 744/5000, epsilon: 0.1, score: 8
Episode: 745/5000, epsilon: 0.1, score: 7
Episode: 746/5000, epsilon: 0.1, score: 9
Episode: 747/5000, epsilon: 0.1, score: 9
Episode: 748/5000, epsilon: 0.1, score: 10
Episode: 749/5000, epsilon: 0.1, score: 7
Episode: 750/5000, epsilon: 0.1, score: 8
Episode: 751/5000, epsilon: 0.1, score: 9
Episode: 752/5000, epsilon: 0.1, score: 9
Episode: 753/5000, epsilon: 0.1, score: 10
Episode: 754/5000, epsilon: 0.1, score: 8
Episode: 755/5000, epsilon: 0.

Episode: 927/5000, epsilon: 0.1, score: 9
Episode: 928/5000, epsilon: 0.1, score: 8
Episode: 929/5000, epsilon: 0.1, score: 8
Episode: 930/5000, epsilon: 0.1, score: 12
Episode: 931/5000, epsilon: 0.1, score: 8
Episode: 932/5000, epsilon: 0.1, score: 8
Episode: 933/5000, epsilon: 0.1, score: 9
Episode: 934/5000, epsilon: 0.1, score: 9
Episode: 935/5000, epsilon: 0.1, score: 10
Episode: 936/5000, epsilon: 0.1, score: 9
Episode: 937/5000, epsilon: 0.1, score: 9
Episode: 938/5000, epsilon: 0.1, score: 9
Episode: 939/5000, epsilon: 0.1, score: 8
Episode: 940/5000, epsilon: 0.1, score: 8
Episode: 941/5000, epsilon: 0.1, score: 8
Episode: 942/5000, epsilon: 0.1, score: 10
Episode: 943/5000, epsilon: 0.1, score: 10
Episode: 944/5000, epsilon: 0.1, score: 11
Episode: 945/5000, epsilon: 0.1, score: 9
Episode: 946/5000, epsilon: 0.1, score: 9
Episode: 947/5000, epsilon: 0.1, score: 8
Episode: 948/5000, epsilon: 0.1, score: 10
Episode: 949/5000, epsilon: 0.1, score: 7
Episode: 950/5000, epsilon: 

Episode: 1119/5000, epsilon: 0.1, score: 8
Episode: 1120/5000, epsilon: 0.1, score: 9
Episode: 1121/5000, epsilon: 0.1, score: 11
Episode: 1122/5000, epsilon: 0.1, score: 10
Episode: 1123/5000, epsilon: 0.1, score: 9
Episode: 1124/5000, epsilon: 0.1, score: 9
Episode: 1125/5000, epsilon: 0.1, score: 9
Episode: 1126/5000, epsilon: 0.1, score: 8
Episode: 1127/5000, epsilon: 0.1, score: 8
Episode: 1128/5000, epsilon: 0.1, score: 8
Episode: 1129/5000, epsilon: 0.1, score: 9
Episode: 1130/5000, epsilon: 0.1, score: 8
Episode: 1131/5000, epsilon: 0.1, score: 8
Episode: 1132/5000, epsilon: 0.1, score: 8
Episode: 1133/5000, epsilon: 0.1, score: 11
Episode: 1134/5000, epsilon: 0.1, score: 8
Episode: 1135/5000, epsilon: 0.1, score: 11
Episode: 1136/5000, epsilon: 0.1, score: 8
Episode: 1137/5000, epsilon: 0.1, score: 10
Episode: 1138/5000, epsilon: 0.1, score: 8
Episode: 1139/5000, epsilon: 0.1, score: 9
Episode: 1140/5000, epsilon: 0.1, score: 12
Episode: 1141/5000, epsilon: 0.1, score: 9
Episo

KeyboardInterrupt: 

# Výsledky

In [ ]:
%matplotlib inline  
import matplotlib.pyplot as plt
import copy

score1 = copy.copy(scores)
score2 = copy.copy(scores)
score3 = copy.copy(scores)

for i in range (len(scores)):
    if i > 1 and i < (len(scores)-2):
        score1[i] = (scores[i - 2] + scores[i - 1] + scores[i] + scores[i + 1] + scores[i + 2])/5
        
for i in range (len(scores)):
    if i > 4 and i < (len(scores)-5):
        score2[i] += scores[i - 5] + scores[i - 4] + scores[i - 3] + scores[i - 2] + scores[i - 1]
        score2[i] += scores[i + 5] + scores[i + 4] + scores[i + 3] + scores[i + 2] + scores[i + 1]
        score2[i] = score2[i]/11

for i in range (len(scores)):
    if i > 49 and i < (len(scores) - 50):
        for e in range (1,50):
            score3[i] += scores[i - e] + scores[i + e] 
        score3[i] = score3[i]/101      

plt.plot(episodesList, scores, 'ro')
plt.ylabel("Score")
plt.xlabel("Episodes")
plt.title("Výsledky")
plt.show()

plt.plot(scores)
plt.ylabel("Score")
plt.xlabel("Episodes")
plt.title("Funkce interpolace výsledků")
plt.show()

plt.plot(score1)
plt.ylabel("Score")
plt.xlabel("Episodes")
plt.title("Funkce interpolace výsledků (filtr - průměr 5-ti prvků)")
plt.show()

plt.plot(score2)
plt.ylabel("Score")
plt.xlabel("Episodes")
plt.title("Funkce interpolace výsledků  (filtr - průměr 11-cti prvků)")
plt.show()

plt.plot(score3)
plt.ylabel("Score")
plt.xlabel("Episodes")
plt.title("Funkce interpolace výsledků  (filtr - průměr 101 prvků)")
plt.show()

saveDQN(net, "./DQN-CartPole-v0.h5")

# Hra

In [ ]:
netForPlay = getDQN(actionCount, stateSize)
loadDQN(netForPlay, "./DQN-CartPole-v0.h5")
observation = env.reset()

for i in range (100):
    state = env.reset()
    state = np.reshape(state, [1, stateSize])
    
    total_reward = 0
    done = False
    
    while not done:
        #env.render()
        Q = netForPlay.predict(state)   
        action = np.argmax(Q)  
        observation_new, reward, done, info = env.step(int(action))

        next_state = np.reshape(observation_new, [1, stateSize])
        state = next_state

        total_reward += reward

    print('{}. Game - score: {}'.format(i, total_reward))